# Index Data

Sheet designed to aggregate all the useful data and store it in a database.

In [104]:
import pandas as pd
import hvplot.pandas
import sqlalchemy
from dotenv import load_dotenv
from pathlib import Path
import requests
import json
import os

load_dotenv('data/.env')
AVAPI = os.getenv('Alpha_Vantage_API')

database_connection_string = 'sqlite:///SP500.db'
engine = sqlalchemy.create_engine(database_connection_string)

### Import SP500  Closing Data

from https://www.spglobal.com/spdji/en/indices/equity/sp-500/#overview

In [105]:
SP500_Data = pd.read_csv(Path("Data/SP500to2012.csv"), index_col='Date', parse_dates=True, infer_datetime_format=True)
SP500_Data.head()

,Close
Date,
2012-01-31,1312.41
2012-02-01,1324.09
2012-02-02,1325.54
2012-02-03,1344.90
2012-02-06,1344.33


### Import Constituent Info

From: https://datahub.io/core/s-and-p-500-companies-financials#resource-constituents-financials

In [106]:
SP500_Constituents_details = pd.read_csv(Path("Data/constituents_financials.csv"), index_col='Symbol')
SP500_Constituents_details = SP500_Constituents_details.sort_values(by=['Market Cap'], ascending=False)
SP500_Constituents_details.head()

,Name,Sector,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book,SEC Filings
Symbol,,,,,,,,,,,,,
AAPL,Apple Inc.,Information Technology,155.15,16.86,1.579541,9.20,180.10,131.1200,8.095080e+11,7.938600e+10,3.458609,5.66,http://www.sec.gov/cgi-bin/browse-edgar?action...
GOOGL,Alphabet Inc Class A,Information Technology,1007.71,31.48,0.000000,22.27,1198.00,824.3000,7.338240e+11,3.421700e+10,6.801692,4.70,http://www.sec.gov/cgi-bin/browse-edgar?action...
GOOG,Alphabet Inc Class C,Information Technology,1001.52,40.29,0.000000,22.27,1186.89,803.1903,7.285360e+11,3.271400e+10,6.772653,4.67,http://www.sec.gov/cgi-bin/browse-edgar?action...
MSFT,Microsoft Corp.,Information Technology,85.01,25.76,1.874791,2.97,96.07,63.2200,6.899780e+11,4.107900e+10,7.113097,9.49,http://www.sec.gov/cgi-bin/browse-edgar?action...
AMZN,Amazon.com Inc,Consumer Discretionary,1350.50,296.16,0.000000,6.16,1498.00,812.5000,6.858730e+11,1.613200e+10,3.927053,24.28,http://www.sec.gov/cgi-bin/browse-edgar?action...


In [77]:
SP500_List = SP500_Constituents_details.index.tolist()

### Import an Individual Stock Ticker

from: Alpha Advantage API https://www.alphavantage.co/documentation/#dailyadj

In [40]:
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&outputsize=full&apikey={AVAPI}'
r = requests.get(url)
data = r.json()
print(json.dumps(data, indent=4, sort_keys=True))

{
    "Meta Data": {
        "1. Information": "Daily Prices (open, high, low, close) and Volumes",
        "2. Symbol": "IBM",
        "3. Last Refreshed": "2022-02-04",
        "4. Output Size": "Full size",
        "5. Time Zone": "US/Eastern"
    },
    "Time Series (Daily)": {
        "1999-11-01": {
            "1. open": "98.5000",
            "2. high": "98.8100",
            "3. low": "96.3700",
            "4. close": "96.7500",
            "5. volume": "9551800"
        },
        "1999-11-02": {
            "1. open": "96.7500",
            "2. high": "96.8100",
            "3. low": "93.6900",
            "4. close": "94.8100",
            "5. volume": "11105400"
        },
        "1999-11-03": {
            "1. open": "95.8700",
            "2. high": "95.9400",
            "3. low": "93.5000",
            "4. close": "94.3700",
            "5. volume": "10369100"
        },
        "1999-11-04": {
            "1. open": "94.4400",
            "2. high": "94.4400",
     

In [47]:
df=pd.DataFrame.from_dict(data['Time Series (Daily)'], orient="index")
df.head()

,1. open,2. high,3. low,4. close,5. volume
2022-02-04,137.8600,138.8200,136.2150,137.1500,4142045
2022-02-03,137.0000,138.7600,135.8310,137.7800,6100777
2022-02-02,135.7000,137.5550,135.2600,137.2500,5357237
2022-02-01,133.7600,135.9600,132.5000,135.5300,6206448
2022-01-31,134.0900,134.0900,132.3000,133.5700,5859043


### Aggregate Constituent DF

Create the table, with AAPL, then append the rest of the table with a for loop

In [117]:
# COMMENTED OUT, no longer needed, used to initilize the database.


# Initial_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=AAPL&outputsize=full&apikey={AVAPI}'
# response = requests.get(Initial_url)
# data = response.json()
# AAPL_df=pd.DataFrame.from_dict(data['Time Series (Daily)'], orient="index")
# AAPL_df = AAPL_df.rename(columns={'4. close':'AAPL'})
# AAPL_df = AAPL_df.drop(columns={'1. open','2. high','3. low','5. volume'})
# df.to_sql('close', engine)
# AAPL_df.head()

In [118]:
return_table = pd.read_sql_table('close', engine)
return_table.head()

,index,AAPL
0,2022-02-04,172.3900
1,2022-02-03,172.9000
2,2022-02-02,175.8400
3,2022-02-01,174.6100
4,2022-01-31,174.7800


C:\Users\canav\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['close']

In [116]:
return_table = pd.read_sql_table('close', engine)
return_table.head()

,index,AAPL
0,2022-02-04,172.3900
1,2022-02-03,172.9000
2,2022-02-02,175.8400
3,2022-02-01,174.6100
4,2022-01-31,174.7800
